## Bad Poets Society - Inspiring Set Collector

This notebook includes the process of collecting poems from "https://www.poetry.com/" . The theme of poetry can be speciliazed.

In [9]:
!pip install requests beautifulsoup4 lxml


In [6]:
!pip install selenium webdriver-manager

In [11]:
import requests
from bs4 import BeautifulSoup
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

def search_poems_selenium(keyword, pages=1):
    results = []
    unique_res = []
    poems  = []

    # Chrome headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")  # 

    # start Selenium ChromeDriver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )  


    for p in range(1, pages + 1):

        if p == 1:
            url = f"https://www.poetry.com/psearch/{keyword}"
        else:
            url = f"https://www.poetry.com/serp.php?st={keyword}&p={p}"
   
        # url = f"https://www.poetry.com/psearch/{keyword}"
        
        print("Opening:", url)
        driver.get(url)
        time.sleep(2)  

        # 
        links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/poem/']")
        # print("Found links:", len(links))

        for link in links:
            href = link.get_attribute("href")
            title = link.text
            if href in unique_res:
                continue
            if href.split("/")[-1]!= keyword:   
                break
            # print("Adding poem:", title, href)
            unique_res.append(href)
            results.append({"title": title, "url": href})

        time.sleep(1)  # be polite

    def _get_soup_from_url_selenium(driver, url):
        driver.get(url)
        time.sleep(2)  
        html = driver.page_source
        return BeautifulSoup(html, "lxml")
    
    # 
    def extract_poem_from_soup(soup):
        poem_div = soup.find(id="disp-quote-body") or soup.select_one("blockquote .poem-body") or soup.select_one("blockquote")
        poem_text = ""
        if poem_div:
            for br in poem_div.find_all("br"):
                br.replace_with("\n")
            poem_text = poem_div.get_text("\n")
            poem_text = "\n".join([line.strip() for line in poem_text.splitlines() if line.strip()])

        return {
            "text": poem_text or ""
        }
    def get_poem_selenium(driver, url):
        soup = _get_soup_from_url_selenium(driver, url)
        return extract_poem_from_soup(soup)
    
    print("Total poems found:", len(results))
    print("Fetching poem texts...")
    for entry in results:
        url = entry["url"]
        poem = get_poem_selenium(driver, url)
        # print("URL:", url)
        # print("Text:\n", poem["text"])
        
        poems.append(poem)
        time.sleep(0.5)  # be polite

    driver.quit()
    return results, poems


In [12]:
# test

keyword = "love"
pages = 3
res, poems = search_poems_selenium(keyword=keyword, pages=pages)
print("Total poems found:", len(poems))
print(res[:5])
# print(poems[:2])

Opening: https://www.poetry.com/psearch/love
Opening: https://www.poetry.com/serp.php?st=love&p=2
Opening: https://www.poetry.com/serp.php?st=love&p=3
Total poems found: 61
Fetching poem texts...
Total poems found: 61
[{'title': '', 'url': 'https://www.poetry.com/poem/111325/love'}, {'title': '', 'url': 'https://www.poetry.com/poem/240241/love'}, {'title': '', 'url': 'https://www.poetry.com/poem/234082/love'}, {'title': '', 'url': 'https://www.poetry.com/poem/143122/love'}, {'title': '', 'url': 'https://www.poetry.com/poem/149226/love'}]


In [ ]:
## Save poems to text file 

import re

punctuation = "，。、！？；:,.!"  
max_words_per_line = 10 

with open(f"poems_{keyword}.txt", "w", encoding="utf-8") as f:
    for i, poem in enumerate(poems, 1):
        #  strip empty lines
        lines = [line.strip() for line in poem['text'].splitlines() if line.strip()]
        # reconstruct text
        text = " ".join(lines)
        # punctuation split lines
        pattern = f"([{punctuation}])"
        text = re.sub(pattern, r"\1\n", text)
        
        # enforce max line length
        final_lines = []
        for line in text.splitlines():
            words = line.split(" ")
            while len(words) > max_words_per_line:
                final_lines.append(" ".join(words[:max_words_per_line]))
                words = words[max_words_per_line:]
            if words:
                final_lines.append(" ".join(words))
        text = "\n".join(final_lines)
        # 
        f.write(f"\n{text}\n\n")